https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a
https://www.browserstack.com/guide/find-element-by-xpath-in-selenium
https://stackoverflow.com/questions/3030487/is-there-a-way-to-get-the-xpath-in-google-chrome
https://www.guru99.com/selenium-webtable.html
https://stackoverflow.com/questions/19200497/python-selenium-webscraping-nosuchelementexception-not-recognized


USe try and catch exceptions to stop iterating and go to the next page or line
https://www.scrapingbee.com/blog/selenium-python/

Selecting from a dropdown
https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python

waiting for page load
https://stackoverflow.com/questions/26566799/wait-until-page-is-loaded-with-selenium-webdriver-for-python
https://www.browserstack.com/guide/selenium-wait-for-page-to-load
https://www.askpython.com/python/examples/python-wait-for-a-specific-time



1. run loop for the column until the end of the row where NoSuchEl exception is trigger
2. have the outer loop move to the next column
3. run loop for the column until the end of the row where NoSuchEl exception is trigger
4. continue until last column which which will tirgger another no such elem exception
5. have selenium click the next page (third outer loop)
6. repeat steps 1-4 until an error is triggered which stops all looping
7. clean up text and send to mySQL db


Header start and end xpaths, use this for the outer loop
Start: //*[@id="pagedResults"]/div[2]/table/thead/tr/th[1]
End: //*[@id="pagedResults"]/div[2]/table/thead/tr/th[17]

Row 1, col 1
//*[@id="pagedResults"]/div[2]/table/tbody/tr[1]/td[1]

Ro1 1, col 2
//*[@id="pagedResults"]/div[2]/table/tbody/tr[1]/td[2]

In [208]:
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
#from collections import defaultdict
import time
import pandas as pd

In [199]:
driver = webdriver.Chrome(r"C:/Users/Charlie/Documents/GitHub/chromedriver.exe")

In [200]:
url = 'https://results.svetiming.com/Santa-Cruz-Triathlon/events/2021/Santa-Cruz-Triathlon/results'
driver.get(url)

In [ ]:
page_num_data = driver.find_element_by_xpath('//*[@id="pagedResults"]/div[1]/div[2]/div/div/span[2]') #look up how many pages
page_num_text = page_num_data.text
page_num_list = page_num_text.split(" ")
page_num = int(page_num_list[1])
print(page_num)

page_num = 2 #overide for debugging
olympic = dict.fromkeys(['Bib','Name', 'Team Name', 'City', 'Start Time', 'Swim', 'T1', 'Bike', 'T2', 'Run', 'Chip Elapsed', 'Division', 'Div Place', 'Age', 'Age Place', 'Gender', 'Gender Place'])
delay = 5

print(type(olympic["Bike"]))

for page in range(0,page_num):
    i = 1
    print(page)
    for key in olympic:
        rows = []
        for r in range(1,55):
            try:
                rows.append(driver.find_element_by_xpath('//*[@id="pagedResults"]/div[2]/table/tbody/tr[' + str(r) + ']/td[' + str(i) + ']').text)
                #print(rows)
            except  NoSuchElementException:
                #print('nse')
                break
        if olympic[key] is None:
            olympic[key]=rows
        else:
            olympic[key].append(rows)
        i=i+1
    #click to the next page of records
    print("im clicking")
    apply = driver.find_element_by_xpath('//*[@id="pagedResults"]/div[1]/div[2]/ul/li[3]/a')
    apply.click()
    print("im waiting")
    #driver.implicitly_wait(10)
    time.sleep(5)
    print("done waiting")

       

print(olympic)

In [ ]:
#get the data for the sprint

catagory = Select(driver.find_element_by_id('SelectedDivisionName'))
catagory.select_by_visible_text('SPRINT-AGE GROUP')

apply = driver.find_element_by_xpath('//*[@id="searchEventResults"]/div/div/span/button')
apply.click()

In [ ]:
event_dict2 = dict.fromkeys(['Bib','Name', 'Team Name', 'City', 'Start Time', 'Swim', 'T1', 'Bike', 'T2', 'Run', 'Chip Elapsed', 'Division', 'Div Place', 'Age', 'Age Place', 'Gender', 'Gender Place'])


i = 1
for key in event_dict2:
    rows = []
    for r in range(1,55):
        try:
            rows.append(driver.find_element_by_xpath('//*[@id="pagedResults"]/div[2]/table/tbody/tr[' + str(r) + ']/td[' + str(i) + ']').text)
            #print(rows)
        except  NoSuchElementException:
            #print('nse')
            break
    event_dict2[key]=rows
    i=i+1





print(event_dict2)

In [ ]:
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pagedResults"]/div[2]/table/thead/tr/th[1]')))
        print ("Page is ready!")
    except TimeoutException:
        print ("Loading took too much time!")
